In [11]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adadelta, Adagrad, Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras import backend as K
from keras.losses import mean_squared_error
from keras.models import load_model, Model
import tensorflow as tf

# Normalize the data.
# from sklearn import preprocessing
from keras.regularizers import l1_l2
import scipy.io as spio
import random

def pass_arg(nsim, tr_size, dropoutrate):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # MC dropout
    class MCDropout(Dropout):
        def call(self, inputs, training=None):
            return super(MCDropout, self).call(inputs, training=True)
    
    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

    #function for computing the density given the temperature(nx1 matrix)
    def density(temp):
        return 1000 * ( 1 - (temp + 288.9414) * (temp - 3.9863)**2 / (508929.2 * (temp + 68.12963) ) )

    def phy_loss_mean(params):
        # useful for cross-checking training
        udendiff, lam = params
        def loss(y_true,y_pred):
            return K.mean(K.relu(udendiff))
        return loss

    #function to calculate the combined loss = sum of rmse and phy based loss
    def combined_loss(params):
        udendiff, lam = params
        def loss(y_true,y_pred):
            return mean_squared_error(y_true, y_pred) + lam * K.mean(K.relu(udendiff))
        return loss

    def PGNN_train_test(optimizer_name, optimizer_val, use_YPhy, pre_train, tr_size, lamda, 
                        iteration, n_nodes, n_layers, drop_frac, reg, lake_num, use_Phy):

    #     fix_seeds(ss)

        # Hyper-parameters of the training process
    #     batch_size = tr_size
        batch_size = 1000
        num_epochs = 1000
        val_frac = 0.2
        patience_val = 100

        # Initializing results filename
        exp_name = "DNN_pre_loss_" + pre_train + optimizer_name + '_trsize' + str(tr_size) + '_lamda' + str(lamda) + '_iter' + str(iteration)
        exp_name = exp_name.replace('.','pt')
        results_dir = '../../results/Lake/'
        model_name = results_dir + exp_name + '_model.h5' # storing the trained model

        # Load features (Xc) and target values (Y)
        data_dir = '../../data/'
        filename = lake_name + '.mat'
        mat = spio.loadmat(data_dir + filename, squeeze_me=True,
        variable_names=['Y','Xc_doy','Modeled_temp'])
        Xc = mat['Xc_doy']
        Y = mat['Y']

        # train and test data
        trainX, trainY = Xc[:tr_size,:-1], Y[:tr_size]
        testX, testY = Xc[-50:,:-1], Y[-50:]
        
        # Loading unsupervised data
        unsup_filename = lake_name + '_sampled.mat'
        unsup_mat = spio.loadmat(data_dir+unsup_filename, squeeze_me=True,
        variable_names=['Xc_doy1','Xc_doy2'])

        uX1 = unsup_mat['Xc_doy1'] # Xc at depth i for every pair of consecutive depth values
        uX2 = unsup_mat['Xc_doy2'] # Xc at depth i + 1 for every pair of consecutive depth values
        uX1 = uX1[:10000,:]
        uX2 = uX2[:10000,:]

        if use_YPhy == 0:
            # Removing the last column from uX (corresponding to Y_PHY)
            uX1 = uX1[:,:-1]
            uX2 = uX2[:,:-1]
            
            
        dependencies = {
         'root_mean_squared_error': root_mean_squared_error
            }

        # load the pre-trained model using non-calibrated physics-based model predictions (./data/unlabeled.dat)
        loaded_model = load_model(results_dir + pre_train, custom_objects=dependencies)

        # Creating the model
        model = Sequential()
        for layer in np.arange(n_layers):
            if layer == 0:
                model.add(Dense(n_nodes, activation='relu', input_shape=(np.shape(trainX)[1],)))
            else:
                if reg:
                    model.add(Dense(n_nodes, activation='relu', kernel_regularizer=l1_l2(l1=.00, l2=.00)))
                else:
                    model.add(Dense(n_nodes, activation='relu'))
            # model.add(Dropout(rate=drop_frac))
            model.add(MCDropout(rate=drop_frac))
        model.add(Dense(1, activation='linear'))

        # pass the weights to all layers but 1st input layer, whose dimensions are updated
        for new_layer, layer in zip(model.layers[1:], loaded_model.layers[1:]):
            new_layer.set_weights(layer.get_weights())

        # physics-based regularization
        uin1 = K.constant(value=uX1) # input at depth i
        uin2 = K.constant(value=uX2) # input at depth i + 1
        lam = K.constant(value=lamda) # regularization hyper-parameter
        uout1 = model(uin1) # model output at depth i
        uout2 = model(uin2) # model output at depth i + 1
        udendiff = (density(uout1) - density(uout2)) # difference in density estimates at every pair of depth values

        totloss = combined_loss([udendiff, lam])
        phyloss = phy_loss_mean([udendiff, lam])


        model.compile(loss=totloss,
                      optimizer=optimizer_val,
                      metrics=[phyloss, root_mean_squared_error])

        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_val, verbose=1)

    #     print('Running...' + optimizer_name)
        history = model.fit(trainX, trainY,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            verbose=1,
                            validation_split=val_frac, callbacks=[early_stopping, TerminateOnNaN()])
    
    
        test_score = model.evaluate(testX, testY, verbose=0)
        print(test_score)

        test_scores = []
        for i in range(int(nsim)):
#             print("simulation num:",i)
#             predictions = model.predict(testX)
#             samples.append(predictions)
            test_score = model.evaluate(testX, testY, verbose=0)
            test_scores.append(test_score[2])
        return np.array(test_scores)
    
    



    # Main Function
    if __name__ == '__main__':

        fix_seeds(1)

        # List of optimizers to choose from    
        optimizer_names = ['Adagrad', 'Adadelta', 'Adam', 'Nadam', 'RMSprop', 'SGD', 'NSGD']
        optimizer_vals = [Adagrad(clipnorm=1), Adadelta(clipnorm=1), Adam(clipnorm=1), Nadam(clipnorm=1), RMSprop(clipnorm=1), SGD(clipnorm=1.), SGD(clipnorm=1, nesterov=True)]

        # selecting the optimizer
        optimizer_num = 2
        optimizer_name = optimizer_names[optimizer_num]
        optimizer_val = optimizer_vals[optimizer_num]

        # Selecting Other Hyper-parameters
        drop_frac = dropoutrate # Fraction of nodes to be dropped out
        use_YPhy = 0 # Whether YPhy is used as another feature in the NN model or not
        n_layers = 2 # Number of hidden layers
        n_nodes = 15 # Number of nodes per hidden layer

        # pre-trained model
        pre_train = 'Lake_Pre-trainAdam_drop0pt1_nL2_nN15_trsize300000_iter0.h5'

        #set lamda
        lamda = 100 # Physics-based regularization constant  
        
        tr_size = int(tr_size)

        # use regularizer
        reg = True


        # total number of runs
        iter_range = np.arange(1)
        
        #List of lakes to choose from
        lake = ['mendota' , 'mille_lacs']
        lake_num = 0  # 0 : mendota , 1 : mille_lacs
        lake_name = lake[lake_num]
        
        testrmse=[]
        # iterating through all possible params
        for iteration in iter_range:
#             results, result_file, pred, obs, rmse, obs_train = PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, 
#                             iteration, n_layers, n_nodes, tr_size, lamda, reg, samp)
#             testrmse.append(rmse)
            pred = PGNN_train_test(optimizer_name, optimizer_val, use_YPhy, 
                                               pre_train, tr_size, lamda, iteration, n_nodes, 
                                   n_layers, drop_frac, reg, lake_name, use_YPhy)
            

    return np.squeeze(pred)

In [12]:
mean_rmses=[]
std_rmses=[]
for ii in ([13450]):
    test_rmse = pass_arg(50, ii, 0.1)
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    mean_rmses.append(mean_rmse)
    std_rmses.append(std_rmse)

Tr_size: 13450
Train on 10760 samples, validate on 2690 samples
Epoch 1/1000
10760/10760 [==============================] - 0s 25us/step - loss: 220.6495 - loss_1: 0.0093 - root_mean_squared_error: 13.4592 - val_loss: 194.6842 - val_loss_1: 0.0093 - val_root_mean_squared_error: 12.0716
Epoch 2/1000
10760/10760 [==============================] - 0s 8us/step - loss: 190.9643 - loss_1: 0.0095 - root_mean_squared_error: 12.2764 - val_loss: 167.9827 - val_loss_1: 0.0086 - val_root_mean_squared_error: 10.9300
Epoch 3/1000
10760/10760 [==============================] - 0s 8us/step - loss: 157.7457 - loss_1: 0.0073 - root_mean_squared_error: 10.9293 - val_loss: 138.7596 - val_loss_1: 0.0077 - val_root_mean_squared_error: 9.7300
Epoch 4/1000
10760/10760 [==============================] - 0s 8us/step - loss: 129.3591 - loss_1: 0.0127 - root_mean_squared_error: 9.5407 - val_loss: 122.1259 - val_loss_1: 0.0216 - val_root_mean_squared_error: 8.7678
Epoch 5/1000
10760/10760 [========================

Epoch 38/1000
10760/10760 [==============================] - 0s 8us/step - loss: 42.3861 - loss_1: 0.0235 - root_mean_squared_error: 4.3167 - val_loss: 45.8683 - val_loss_1: 0.0237 - val_root_mean_squared_error: 4.6282
Epoch 39/1000
10760/10760 [==============================] - 0s 8us/step - loss: 41.9591 - loss_1: 0.0232 - root_mean_squared_error: 4.2866 - val_loss: 44.4932 - val_loss_1: 0.0224 - val_root_mean_squared_error: 4.6036
Epoch 40/1000
10760/10760 [==============================] - 0s 8us/step - loss: 41.8225 - loss_1: 0.0225 - root_mean_squared_error: 4.2768 - val_loss: 44.5218 - val_loss_1: 0.0226 - val_root_mean_squared_error: 4.5899
Epoch 41/1000
10760/10760 [==============================] - 0s 8us/step - loss: 41.1962 - loss_1: 0.0220 - root_mean_squared_error: 4.2384 - val_loss: 43.9594 - val_loss_1: 0.0215 - val_root_mean_squared_error: 4.5539
Epoch 42/1000
10760/10760 [==============================] - 0s 8us/step - loss: 40.4668 - loss_1: 0.0223 - root_mean_square

10760/10760 [==============================] - 0s 8us/step - loss: 33.1370 - loss_1: 0.0144 - root_mean_squared_error: 3.7648 - val_loss: 25.7344 - val_loss_1: 0.0147 - val_root_mean_squared_error: 3.1801
Epoch 76/1000
10760/10760 [==============================] - 0s 8us/step - loss: 32.8579 - loss_1: 0.0144 - root_mean_squared_error: 3.7340 - val_loss: 25.6706 - val_loss_1: 0.0138 - val_root_mean_squared_error: 3.1929
Epoch 77/1000
10760/10760 [==============================] - 0s 8us/step - loss: 32.7776 - loss_1: 0.0145 - root_mean_squared_error: 3.7393 - val_loss: 25.4417 - val_loss_1: 0.0153 - val_root_mean_squared_error: 3.1371
Epoch 78/1000
10760/10760 [==============================] - 0s 8us/step - loss: 32.8262 - loss_1: 0.0146 - root_mean_squared_error: 3.7325 - val_loss: 25.6943 - val_loss_1: 0.0146 - val_root_mean_squared_error: 3.1587
Epoch 79/1000
10760/10760 [==============================] - 0s 8us/step - loss: 32.7708 - loss_1: 0.0146 - root_mean_squared_error: 3.725

10760/10760 [==============================] - 0s 8us/step - loss: 28.4817 - loss_1: 0.0126 - root_mean_squared_error: 3.4174 - val_loss: 20.3915 - val_loss_1: 0.0126 - val_root_mean_squared_error: 2.7533
Epoch 150/1000
10760/10760 [==============================] - 0s 8us/step - loss: 28.5884 - loss_1: 0.0126 - root_mean_squared_error: 3.4359 - val_loss: 20.9738 - val_loss_1: 0.0130 - val_root_mean_squared_error: 2.8035
Epoch 151/1000
10760/10760 [==============================] - 0s 8us/step - loss: 28.1714 - loss_1: 0.0125 - root_mean_squared_error: 3.4119 - val_loss: 20.3396 - val_loss_1: 0.0124 - val_root_mean_squared_error: 2.7714
Epoch 152/1000
10760/10760 [==============================] - 0s 8us/step - loss: 28.6984 - loss_1: 0.0129 - root_mean_squared_error: 3.4452 - val_loss: 20.4409 - val_loss_1: 0.0126 - val_root_mean_squared_error: 2.7501
Epoch 153/1000
10760/10760 [==============================] - 0s 8us/step - loss: 28.1154 - loss_1: 0.0127 - root_mean_squared_error: 3

10760/10760 [==============================] - 0s 8us/step - loss: 25.9621 - loss_1: 0.0119 - root_mean_squared_error: 3.2355 - val_loss: 19.2248 - val_loss_1: 0.0117 - val_root_mean_squared_error: 2.7209
Epoch 224/1000
10760/10760 [==============================] - 0s 8us/step - loss: 26.0176 - loss_1: 0.0119 - root_mean_squared_error: 3.2397 - val_loss: 19.5001 - val_loss_1: 0.0120 - val_root_mean_squared_error: 2.7028
Epoch 225/1000
10760/10760 [==============================] - 0s 8us/step - loss: 25.8492 - loss_1: 0.0117 - root_mean_squared_error: 3.2421 - val_loss: 19.4669 - val_loss_1: 0.0121 - val_root_mean_squared_error: 2.7197
Epoch 226/1000
10760/10760 [==============================] - 0s 8us/step - loss: 25.6202 - loss_1: 0.0121 - root_mean_squared_error: 3.2080 - val_loss: 19.7195 - val_loss_1: 0.0117 - val_root_mean_squared_error: 2.7260
Epoch 227/1000
10760/10760 [==============================] - 0s 8us/step - loss: 25.5207 - loss_1: 0.0118 - root_mean_squared_error: 3

...

Epoch 704/1000
10760/10760 [==============================] - 0s 7us/step - loss: 15.1901 - loss_1: 0.0073 - root_mean_squared_error: 2.4215 - val_loss: 13.9125 - val_loss_1: 0.0074 - val_root_mean_squared_error: 2.3401
Epoch 705/1000
10760/10760 [==============================] - 0s 8us/step - loss: 15.4140 - loss_1: 0.0072 - root_mean_squared_error: 2.4678 - val_loss: 13.4445 - val_loss_1: 0.0074 - val_root_mean_squared_error: 2.3023
Epoch 706/1000
10760/10760 [==============================] - 0s 8us/step - loss: 15.2170 - loss_1: 0.0073 - root_mean_squared_error: 2.4220 - val_loss: 13.8301 - val_loss_1: 0.0075 - val_root_mean_squared_error: 2.3407
Epoch 707/1000
10760/10760 [==============================] - 0s 7us/step - loss: 15.4682 - loss_1: 0.0073 - root_mean_squared_error: 2.4738 - val_loss: 13.7353 - val_loss_1: 0.0069 - val_root_mean_squared_error: 2.3505
Epoch 708/1000
10760/10760 [==============================] - 0s 7us/step - loss: 14.9995 - loss_1: 0.0074 - root_mean_s

10760/10760 [==============================] - 0s 8us/step - loss: 14.2472 - loss_1: 0.0070 - root_mean_squared_error: 2.3308 - val_loss: 12.6080 - val_loss_1: 0.0070 - val_root_mean_squared_error: 2.2341
Epoch 779/1000
10760/10760 [==============================] - 0s 7us/step - loss: 14.2544 - loss_1: 0.0069 - root_mean_squared_error: 2.3326 - val_loss: 13.0146 - val_loss_1: 0.0070 - val_root_mean_squared_error: 2.2701
Epoch 780/1000
10760/10760 [==============================] - 0s 8us/step - loss: 13.9391 - loss_1: 0.0068 - root_mean_squared_error: 2.3175 - val_loss: 12.6975 - val_loss_1: 0.0066 - val_root_mean_squared_error: 2.2394
Epoch 781/1000
10760/10760 [==============================] - 0s 8us/step - loss: 14.2630 - loss_1: 0.0069 - root_mean_squared_error: 2.3558 - val_loss: 13.5762 - val_loss_1: 0.0069 - val_root_mean_squared_error: 2.3060
Epoch 782/1000
10760/10760 [==============================] - 0s 7us/step - loss: 14.0851 - loss_1: 0.0070 - root_mean_squared_error: 2

Epoch 815/1000
10760/10760 [==============================] - 0s 8us/step - loss: 14.0042 - loss_1: 0.0068 - root_mean_squared_error: 2.3125 - val_loss: 12.4163 - val_loss_1: 0.0065 - val_root_mean_squared_error: 2.2304
Epoch 816/1000
10760/10760 [==============================] - 0s 7us/step - loss: 13.8439 - loss_1: 0.0067 - root_mean_squared_error: 2.3060 - val_loss: 12.8834 - val_loss_1: 0.0066 - val_root_mean_squared_error: 2.3023
Epoch 817/1000
10760/10760 [==============================] - 0s 8us/step - loss: 13.8551 - loss_1: 0.0066 - root_mean_squared_error: 2.3245 - val_loss: 12.5770 - val_loss_1: 0.0069 - val_root_mean_squared_error: 2.2627
Epoch 818/1000
10760/10760 [==============================] - 0s 8us/step - loss: 13.6243 - loss_1: 0.0068 - root_mean_squared_error: 2.2931 - val_loss: 12.7517 - val_loss_1: 0.0066 - val_root_mean_squared_error: 2.2567
Epoch 819/1000
10760/10760 [==============================] - 0s 8us/step - loss: 13.7174 - loss_1: 0.0066 - root_mean_s

10760/10760 [==============================] - 0s 8us/step - loss: 13.2094 - loss_1: 0.0064 - root_mean_squared_error: 2.2477 - val_loss: 12.0061 - val_loss_1: 0.0066 - val_root_mean_squared_error: 2.1892
Epoch 890/1000
10760/10760 [==============================] - 0s 7us/step - loss: 13.1699 - loss_1: 0.0068 - root_mean_squared_error: 2.2488 - val_loss: 12.7152 - val_loss_1: 0.0065 - val_root_mean_squared_error: 2.2920
Epoch 891/1000
10760/10760 [==============================] - 0s 8us/step - loss: 12.9739 - loss_1: 0.0065 - root_mean_squared_error: 2.2459 - val_loss: 11.8992 - val_loss_1: 0.0064 - val_root_mean_squared_error: 2.2306
Epoch 892/1000
10760/10760 [==============================] - 0s 7us/step - loss: 13.0034 - loss_1: 0.0065 - root_mean_squared_error: 2.2248 - val_loss: 12.3759 - val_loss_1: 0.0064 - val_root_mean_squared_error: 2.2524
Epoch 893/1000
10760/10760 [==============================] - 0s 8us/step - loss: 13.3856 - loss_1: 0.0066 - root_mean_squared_error: 2

Epoch 963/1000
10760/10760 [==============================] - 0s 7us/step - loss: 12.6702 - loss_1: 0.0065 - root_mean_squared_error: 2.1957 - val_loss: 11.8591 - val_loss_1: 0.0063 - val_root_mean_squared_error: 2.2067
Epoch 964/1000
10760/10760 [==============================] - 0s 7us/step - loss: 12.7942 - loss_1: 0.0065 - root_mean_squared_error: 2.2078 - val_loss: 11.8431 - val_loss_1: 0.0064 - val_root_mean_squared_error: 2.2040
Epoch 965/1000
10760/10760 [==============================] - 0s 7us/step - loss: 12.8193 - loss_1: 0.0065 - root_mean_squared_error: 2.2280 - val_loss: 12.2529 - val_loss_1: 0.0065 - val_root_mean_squared_error: 2.2293
Epoch 966/1000
10760/10760 [==============================] - 0s 8us/step - loss: 12.7891 - loss_1: 0.0064 - root_mean_squared_error: 2.2245 - val_loss: 12.1086 - val_loss_1: 0.0065 - val_root_mean_squared_error: 2.1891
Epoch 967/1000
10760/10760 [==============================] - 0s 8us/step - loss: 12.6806 - loss_1: 0.0066 - root_mean_s

In [13]:
mean_rmses

[3.17712158203125]

In [8]:
std_rmses

[0.002223141053603035,
 0.001305800732591835,
 0.0006557760316852748,
 0.0006305779694075574,
 0.0009157213424093879]

In [10]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mean_rmses, "../mean_rmse_dnn_updloss_MC.dat")
save_obj(std_rmses, "../std_rmse_dnn_updloss_MC.dat")


In [2]:
pred = pass_arg(50, 20)

Tr_size: 20
19/19 [==============================] - 0s 52us/step
[0.015291403979063034, 0.0011981830466538668, 0.08771183341741562]


In [3]:
np.std(pred,axis=0)**2

array([3.37335584e-03, 1.94856222e-03, 4.34233528e-03, 1.79976993e-03,
       1.79456628e-03, 2.80441530e-03, 1.06975134e-03, 1.08144968e-03,
       2.36601476e-03, 2.80755199e-03, 9.52134375e-04, 7.28175917e-04,
       8.28316202e-04, 1.34716392e-03, 7.11925852e-04, 1.69787777e-03,
       1.15746888e-03, 1.34285321e-04, 1.15999870e-03, 3.23958276e-03,
       8.84820300e-04, 9.76184907e-04, 2.75417347e-03, 1.36949599e-03,
       3.28332465e-03, 1.53482391e-03, 1.64989592e-03, 7.30333093e-04,
       5.57333988e-04, 7.09685683e-03, 1.49872701e-03, 1.56181736e-03,
       2.13612244e-03, 8.86846101e-04, 9.92977992e-04, 2.32536369e-03,
       1.08897465e-03, 1.44896924e-03, 7.44325982e-04, 2.09604646e-03,
       2.44449568e-03, 1.82411820e-03, 3.52882384e-03, 2.68475548e-03,
       2.96447333e-03, 1.25102268e-03, 1.98329519e-03, 3.01357242e-03,
       1.52132928e-03, 1.97036774e-03, 3.22001870e-04, 1.20475620e-03,
       1.21259701e-03, 1.74869748e-03, 2.93545797e-03, 5.46205556e-03,
      

In [4]:
mc_pred=np.mean(pred,axis=0)
mc_pred

array([0.0431564 , 0.04969775, 0.03855895, 0.0473083 , 0.04825364,
       0.04153984, 0.03876032, 0.02742871, 0.05394102, 0.03834876,
       0.05311058, 0.04025944, 0.04402751, 0.0490651 , 0.04529456,
       0.05169499, 0.05430209, 0.03668012, 0.05224204, 0.03989543,
       0.05179229, 0.05054558, 0.04952174, 0.05267722, 0.03513261,
       0.05475615, 0.0515241 , 0.03853419, 0.02725641, 0.05868765,
       0.05132379, 0.06161826, 0.044791  , 0.05282198, 0.05968352,
       0.036227  , 0.04257078, 0.04440696, 0.050751  , 0.05052431,
       0.03119921, 0.05248038, 0.05077071, 0.05365586, 0.04209185,
       0.04676313, 0.04421702, 0.04579834, 0.04491324, 0.05062904,
       0.01135702, 0.05096268, 0.0548835 , 0.04697408, 0.04883337,
       0.02582511, 0.03095761, 0.03505554, 0.05255293, 0.04615692,
       0.04236671, 0.04940849, 0.05350411, 0.05224761, 0.04388189,
       0.04533507, 0.0102862 , 0.04446835, 0.04741205, 0.05951528,
       0.05657636, 0.02142219, 0.03366445, 0.04343371, 0.05729

In [5]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mc_pred, "../pred_upd_loss_MC_Xx.dat")